In [1]:
import os
from datetime import datetime

drive = True

if drive:
    from google.colab import drive
    drive.mount('/content/drive')

    %cd /content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob

    path = "/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob/"

    # check python file folder
    assert os.path.exists(path + "python"), f"Upload python files in {path}python"
    %cd ./python

    # check data folder
    assert os.path.exists(path + "data"), f"Upload data files in {path}data"
else:
    path = "../"

    # check python file folder
    assert os.path.exists(path + "python"), f"Upload python files in {path}python"

    # check data folder
    assert os.path.exists(path + "data"), f"Upload data files in {path}data"

Mounted at /content/drive
/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob
/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob/python


In [2]:
import myk_data_par
import myk_models
import myk_loss
import myk_train
import myk_evaluate_par

import torch
from torch.utils.data import DataLoader

In [3]:
from torch.utils.tensorboard import SummaryWriter

In [7]:
# used for the writing of example outputs
run_name="audio_logic_dist_par"
# dataset : need an input and output folder in this folder
audio_folder = f"../data/{run_name}"
#audio_folder = "../../data/audio_ht1"
assert os.path.exists(audio_folder), f"Audio folder  not found. Looked for {audio_folder}"
# used to render example output during training
test_file = "../data/guitar.wav"
assert os.path.exists(test_file), "Test file not found. Looked for " + test_file

In [8]:
# initialize net specs
lstm_hidden_size = 32
learning_rate = 5e-3
batch_size = 50
max_epochs = 10000

# create the logger for tensorboard
writer = SummaryWriter()

In [9]:
print("Loading dataset from folder ", audio_folder)
dataset = myk_data_par.generate_dataset(audio_folder + "/input/", audio_folder + "/output/", frag_len_seconds=0.5)

print("Splitting dataset")
train_ds, val_ds, test_ds = myk_data_par.get_train_valid_test_datasets(dataset)

Loading dataset from folder  ../data/audio_logic_dist_par
loading input and output of logicdist
    loading output of logicdist with parameter 0
    generate_dataset:: Loaded frames from audio file 22050
    found input fragments of shape (120, 22050, 2)
    found output fragments of shape (120, 22050, 1)
    total input shape: (120, 22050, 2)
    total output shape: (120, 22050, 1)

    loading output of logicdist with parameter 0.33
    generate_dataset:: Loaded frames from audio file 22050
    found input fragments of shape (120, 22050, 2)
    found output fragments of shape (120, 22050, 1)
    total input shape: (240, 22050, 2)
    total output shape: (240, 22050, 1)

    loading output of logicdist with parameter 0.66
    generate_dataset:: Loaded frames from audio file 22050
    found input fragments of shape (120, 22050, 2)
    found output fragments of shape (120, 22050, 1)
    total input shape: (360, 22050, 2)
    total output shape: (360, 22050, 1)

    loading output of log

In [10]:
dataset[0]

(tensor([[-6.1035e-05,  0.0000e+00],
         [-1.8311e-04,  0.0000e+00],
         [-1.8311e-04,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00]]),
 tensor([[-3.0518e-05],
         [-6.1035e-05],
         [-6.1035e-05],
         ...,
         [ 0.0000e+00],
         [ 0.0000e+00],
         [ 0.0000e+00]]))

In [11]:
# test GPU, must be done after splitting
device = myk_train.get_device()

cuda device available


In [12]:
# create data loaders
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))

In [13]:
model = myk_models.SimpleLSTM(hidden_size=lstm_hidden_size, param=True).to(device)
print(model)

SimpleLSTM(
  (lstm): LSTM(2, 32, batch_first=True)
  (dense): Linear(in_features=32, out_features=1, bias=True)
)


In [14]:
# crate optimizer and loss function
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', factor=0.5, patience=5, verbose=True) non lo usava

loss_functions = myk_loss.LossWrapper()

# https://github.com/Alec-Wright/Automated-GuitarAmpModelling/blob/main/dist_model_recnet.py
# https://github.com/Alec-Wright/CoreAudioML/blob/bad9469f94a2fa63a50d70ff75f5eff2208ba03f/training.py

In [15]:
#%load_ext tensorboard
#%tensorboard --logdir logs

In [16]:
# training loop
lowest_val_loss = 0
best_loss = False

max_patience = 500
curr_patience = max_patience

# datetime object containing current date and time
best_epoch = 0
now = datetime.now()
dt_string = now.strftime(f"models/{model.model_type}_%d-%m-%Y_%H-%M-%S")

os.mkdir(os.path.join(".", dt_string))

for epoch in range(max_epochs):
    #ep_loss = myk_train.train_epoch_interval(model, train_dl, loss_functions, optimiser, device=device)
    ep_loss = myk_train.train_epoch_interval(model, train_dl, loss_functions, optimiser, device=device)

    #ep_loss = myk_train.train_epoch(model, train_dl, loss_functions, optimiser, device=device)
    val_loss = myk_train.compute_batch_loss(model, val_dl, loss_functions, device=device)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Loss/train", ep_loss, epoch)

    # check if we have beaten our best loss to date
    if lowest_val_loss == 0:# first run
        lowest_val_loss = val_loss
    elif val_loss < lowest_val_loss:# new record
        lowest_val_loss = val_loss
        best_loss = True
    else: # no improvement
        best_loss = False
        curr_patience -= 1

    if best_loss: # save best model so far
        best_epoch = epoch
        print(f"    Record loss - saving at epoch {epoch}")
        # save for RTNeural
        model.save_for_rtneural(f"{dt_string}/model.json")
        # save for pythorch
        torch.save(model.state_dict(), f"{dt_string}/model.ph")
        print(f"    epoch {epoch}, train_loss {ep_loss}, val_loss {val_loss} ")
        curr_patience = max_patience
    if epoch % 50 == 0: # save an example processed audio file
        myk_evaluate_par.run_file_through_model(model, test_file, audio_folder + "/" + run_name + str(epoch)+".wav")
        print(f"epoch {epoch}, train_loss {ep_loss}, val_loss {val_loss} ")
    if curr_patience == 0:
        print("max patience reached, stopping training")
        # load best parameters in the model
        model.load_state_dict(torch.load(f"{dt_string}/model.ph"))
        model.eval() # set inference state in the possible layers that need it
        myk_evaluate_par.run_file_through_model(model, test_file, audio_folder + "/" + run_name + str(best_epoch)+"_BEST.wav", final_eval=True)
        break

epoch 0, train_loss 0.29605332016944885, val_loss 0.08199474215507507 
    Record loss - saving at epoch 1
    epoch 1, train_loss 0.05558342859148979, val_loss 0.04412418603897095 
    Record loss - saving at epoch 2
    epoch 2, train_loss 0.030858855694532394, val_loss 0.01930098980665207 
    Record loss - saving at epoch 3
    epoch 3, train_loss 0.012693321332335472, val_loss 0.007782535161823034 
    Record loss - saving at epoch 4
    epoch 4, train_loss 0.006805948913097382, val_loss 0.006514872424304485 
    Record loss - saving at epoch 6
    epoch 6, train_loss 0.006079407874494791, val_loss 0.00468940706923604 
    Record loss - saving at epoch 7
    epoch 7, train_loss 0.004635797347873449, val_loss 0.004467559512704611 
    Record loss - saving at epoch 8
    epoch 8, train_loss 0.00436056312173605, val_loss 0.004078370984643698 
    Record loss - saving at epoch 10
    epoch 10, train_loss 0.004094716627150774, val_loss 0.003717538435012102 
    Record loss - saving at 

In [17]:
to_append = [f'\n{run_name}/*.wav', f'\n!{run_name}/*_BEST*.wav']

with open(f'{audio_folder}/../.gitignore', "r+") as gitignore:
    read = gitignore.read()
    if to_append[0][2:len(to_append[0])] not in read:
        gitignore.writelines(to_append)